The SQLite file "Edinburgh_Buses.db" describes the routes on the buses in the Edinburgh (Scotland) area at some time in the past. These routes were operated by quite a few different bus companies.

It contains two tables: "route" which contains the following fields:

1. "num" which is the route number (this sometimes also has some letters in it, like "46A")
2. "company" which is the bus company name
3. "pos" which is the position of the stop given by "stop" along the route
4. "stop" which is the stop number

and "stops" which contains:

1. "id" which is the stop number (joins with "stop" in the other table)
2. "stop name"

Note that two companies can use the same route number and those are different routes. For instance, in the data, companies "LRT" and "SMT" both operate a route 16.

The 11 questions on this assignment can be found below the startup code below. Please embed new code cells below each question. Please hand in a .ipynb file which can be exported from Colab. Each question is worth 3 points. 

You may need to change the mount point of the SQLite file to your own Google Drive, and put the SQLite database file in your own Google Drive, in order to make the notebook work.



**Carlos Figueroa,
New York University, Spring 2022,
Data Management,
Assignment #2**

In [94]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [95]:
# if you do not have write access to this path, you may need 
# to change it to a path that you do have write access to.
gdrivePath="/content/drive/MyDrive/Bhj/"
import sqlite3
import pandas as pd
# create a connection to a database
conn = sqlite3.connect(gdrivePath + "Edinburgh_Buses.db") 

In [3]:
def execSQL(conn,query):
  conn.execute(query) # execute an SQL query
  conn.commit() # "commit" that query in order to make its action permanent

In [4]:
def allrowsSelect(conn,query):
  cursor = conn.execute(query)
  for row in cursor:
   print(row)

In [5]:
# sample query
import pandas as pd
pd.read_sql_query('SELECT * FROM route WHERE company="LRT"',conn)

,num,company,pos,stop
0,1,LRT,1,137
1,1,LRT,2,99
2,1,LRT,3,59
3,1,LRT,4,66
4,1,LRT,5,42
...,...,...,...,...
687,9,LRT,5,37
688,9,LRT,6,99
689,9,LRT,7,105
690,9,LRT,8,162


In [6]:
# sample query
import pandas as pd
pd.read_sql_query('SELECT * FROM stops',conn)

,id,name
0,1,Aberlady
1,2,Abington
2,3,Amisfield Park
3,4,Ancrum
4,5,Armadale
...,...,...
241,246,Whitburn
242,247,Whitecraig
243,248,Wilkieston
244,249,Willowbrae


1. Write a query to find all of the distinct bus companies.

In [7]:
pd.read_sql_query('SELECT DISTINCT company FROM route',conn)

,company
0,LRT
1,MAC
2,SMT
3,SMJ
4,CCC
5,MID
6,LOW
7,LRJ
8,WIL
9,FS


2. Write a query that lists all of the distinct route number and company combinations.

In [8]:
pd.read_sql_query('SELECT DISTINCT company, num FROM route',conn) #all the distinct values from the num column

,company,num
0,LRT,1
1,LRT,10
2,LRT,100
3,MAC,100
4,SMT,106
...,...,...
127,SMT,D27
128,SMT,D28
129,ROU,R1
130,ROU,R2


3. Find a query to find the route numbers that are used by more than one company, and the number of times that they are used.

In [107]:
query='''
SELECT DISTINCT num, count(distinct company) as number_companies_using, count(num) as number_of_times_used
FROM route
GROUP BY num
HAVING number_companies_using>1;
'''
pd.read_sql_query(query,conn)

,num,number_companies_using,number_of_times_used
0,100,2,12
1,16,2,17
2,19,2,19
3,27,2,15
4,30,2,20
5,37,2,15
6,38,2,21
7,43,2,18
8,47,2,17
9,55,2,12


4. Write a query to find the names of all of the stops on route 17 operated by company LRT, ordered by position. Output the position, stop number, and stop name.

In [10]:
query='''
SELECT pos, stop, num, name
FROM route
LEFT OUTER JOIN stops on route.stop = stops.id
WHERE num == 17
'''
pd.read_sql_query(query,conn)

,pos,stop,num,name
0,1,167,17,Newhaven
1,2,137,17,Leith
2,3,141,17,Leith Walk
3,4,194,17,Princes Street
4,5,230,17,Tollcross
5,6,160,17,Morningside
6,7,179,17,Oxgangs


5. Write a query that counts how many distinct routes each company has. It should output the company name and the number of routes.

In [110]:
query='''
SELECT distinct company, count(distinct num) as distinct_route_number
FROM route
GROUP BY company
'''
pd.read_sql_query(query,conn)

,company,distinct_route_number
0,CCC,1
1,FF,1
2,FS,2
3,JM,1
4,LOW,6
5,LRJ,1
6,LRT,72
7,MAC,1
8,MID,4
9,ROU,2


6. Write a query to find all of the distinct routes operated by company "LOW".

In [12]:
query='''
SELECT distinct company, count(num) as number_of_routes,  count(distinct num) as distinct_routes
FROM route
WHERE company == "LOW"
'''
pd.read_sql_query(query,conn)

,company,number_of_routes,distinct_routes
0,LOW,50,6


In [116]:
#or just print the distinct routes in a list
query='''
select distinct num, company 
from route
where company == "LOW"
'''
pd.read_sql_query(query,conn)

,num,company
0,29,LOW
1,30,LOW
2,315,LOW
3,62,LOW
4,94,LOW
5,95,LOW


7. Find the ten routes with the most stops. Remember that a route is identified by both its stop and number.

In [119]:
#if route depends of num and company
query='''
SELECT DISTINCT num, company, count(stop) as different_stops
FROM route 
GROUP BY num, company
ORDER BY different_stops DESC
LIMIT 10
'''
pd.read_sql_query(query,conn)
#some stops are being repeated, so if we use distinct inside count of stop, we will get a different result

,num,company,different_stops
0,32,LRT,23
1,52,LRT,23
2,12,LRT,18
3,2A,LRT,18
4,2,LRT,17
5,34,LRT,15
6,35,LRT,15
7,66A,SMT,13
8,81A,LRT,13
9,14,LRT,12


8. Find the ten routes with the fewest stops.

In [16]:
#if route depends of num and company
query='''
SELECT DISTINCT num, company, count(stop) as different_stops
FROM route 
GROUP BY num, company
ORDER BY different_stops ASC
LIMIT 10
'''
pd.read_sql_query(query,conn)

,num,company,different_stops
0,60,JM,2
1,59,LRT,3
2,100,LRT,4
3,48,WIL,4
4,55,LRT,4
5,86A,LRT,4
6,93,SMT,4
7,VL2,VL,4
8,49,SMT,5
9,58,LRT,5


9. Do any routes have the same stop on them more than once? Write a query to check if they do, by creating a table containing the num, company, and stop columns for all of the stops that appear more than one, along with the number of times that they appear.

In [50]:
#if route depends of num and company
query='''
SELECT DISTINCT num, company, stop, count(*) as number_of_times_repeated_on_route
FROM route
GROUP BY num, company, stop
HAVING count(*)>1
LIMIT 15
'''
pd.read_sql_query(query,conn)

,num,company,stop,number_of_times_repeated_on_route
0,1,LRT,137,2
1,12,LRT,31,2
2,12,LRT,115,2
3,12,LRT,173,2
4,14,LRT,215,2
5,19,LRT,28,2
6,19A,LRT,28,2
7,2,LRT,31,2
8,2,LRT,115,2
9,2,LRT,173,2


In [48]:
#we can check that the table on top is true by looking at this
#this is for proving, please disregard
query='''
select *, (number_of_total_stops - distinct_stops) as repetitions
from
(select num, company, GROUP_CONCAT(stop) as stops, GROUP_CONCAT(distinct stop) as dstops, count(stop) as number_of_total_stops, count(distinct stop) as distinct_stops
from route 
group by num, company)
order by repetitions DESC 
limit 10
'''
pd.read_sql_query(query,conn)

,num,company,stops,dstops,number_of_total_stops,distinct_stops,repetitions
0,81,LRT,"67,184,142,29,219,47,59,162,219,47,59,162","67,184,142,29,219,47,59,162",12,8,4
1,81A,LRT,"67,184,146,142,29,219,47,59,162,219,47,59,162","67,184,146,142,29,219,47,59,162",13,9,4
2,12,LRT,"173,54,192,107,115,101,223,31,109,31,48,115,19...","173,54,192,107,115,101,223,31,109,48,194,141,1...",18,15,3
3,2,LRT,"173,191,50,137,141,194,115,48,31,31,223,101,11...","173,191,50,137,141,194,115,48,31,223,101,107,1...",17,14,3
4,2A,LRT,"173,191,50,137,141,194,115,48,31,109,31,223,10...","173,191,50,137,141,194,115,48,31,109,223,101,1...",18,15,3
5,275,SMT,"219,48,165,48,165,32,236,195,145","219,48,165,32,236,195,145",9,7,2
6,34,LRT,"213,101,92,39,225,37,26,137,147,149,176,116,92...","213,101,92,39,225,37,26,137,147,149,176,116,152",15,13,2
7,35,LRT,"213,152,92,116,176,149,147,137,26,37,225,39,92...","213,152,92,116,176,149,147,137,26,37,225,39,101",15,13,2
8,1,LRT,"137,99,59,66,42,48,223,92,205,80,137","137,99,59,66,42,48,223,92,205,80",11,10,1
9,14,LRT,"215,162,28,99,137,141,29,192,54,173,166,215","215,162,28,99,137,141,29,192,54,173,166",12,11,1


10. Find the numbers and names of the ten stops that are on the most routes, along with the number of routes they are on. Count a stop only once for each route it appears on. For instance, stop 137 appears both at the beginning (position 1) and the end (position 11) of route 1 of company LRT; stop 137 should be counted only once for this route.

In [120]:
#if route depends of num and company
query='''
SELECT * , count(stop) as num_routes
FROM
(SELECT num, company, name, pos, stop
FROM route
LEFT OUTER JOIN stops on route.stop = stops.id
GROUP by num, company, pos)
GROUP by stop
ORDER by num_routes DESC
LIMIT 10
'''
pd.read_sql_query(query,conn)

,num,company,name,pos,stop,num_routes
0,D28,SMT,St Andrew Square,1,219,49
1,C70,SMT,Haymarket,7,115,45
2,C70,SMT,Princes Street,8,194,38
3,C70,SMT,Bridges,9,29,36
4,C12,SMT,Corstorphine,3,48,28
5,C55,SMT,London Road,6,149,25
6,C5,SMT,Leith,9,137,22
7,D28,SMT,Sighthill,2,213,22
8,C70,SMT,Gorgie,6,101,20
9,C33,SMT,Musselburgh,9,163,20


11. Find all of the routes that go from stop 219 to stop 115. The position of stop 219 on the route needs to be before that of 115.

In [124]:
#since we know that the previous of 115 has to be 219 to be counted as a different route, we then have
query='''
SELECT distinct a.num, a.company, a.stop, b.stop as previous_stop
FROM route as a
INNER JOIN route as b on a.num = b.num and a.company = b.company
WHERE a.stop = 115 and previous_stop = 219
'''
pd.read_sql_query(query,conn)

,num,company,stop,previous_stop
0,137,SMT,115,219
1,22,LRT,115,219
2,22A,LRT,115,219
3,25,LRT,115,219
4,37,SMT,115,219
5,38,MID,115,219
6,61,LRT,115,219
7,61,SMT,115,219
8,63,LRT,115,219
9,C12,SMT,115,219
